### Header Code

In [25]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
import plotly.graph_objs as go
import math
import time
from scipy.optimize import fmin
from scipy.optimize import minimize

## BAPM Functions

In [8]:
#generates the stock price vector for the BAPM function
def generate_stockPrices(u, d, row, col, matrix):
    if row > col:
        return matrix
    if ((row == 0) and (col == 0)):
                matrix[row][col] = 1
    else:
        matrix[row][col] = u**(col - row) * d**(row)
    return matrix

In [9]:
#generates the state price vector for the BAPM function
def generate_statePrices(q, row, col, matrix):
    if col == row:
        matrix[row][col] = q    
    if col - row == -1:
        matrix[col][row] = 1-q
    return matrix

In [10]:
def generate_finalValue(call, strikePrice, value):
    if call:
        value = value - strikePrice
    else:
        value = strikePrice - value
    return np.clip(value, a_min=0, a_max=None)

In [14]:
def generate_optionPrices(optionType, strikePrice, valueMatrix, q, discount):
    #print(valueMatrix.shape[1])
    for j in range(valueMatrix.shape[1]-1, 0, -1):#loop backwards through columns first
        for i in range(1, valueMatrix.shape[0]):#loop downwards through rows; saves final spot of [0,0]
            if i > j:
                continue
            if not optionType:#european option pricing
                valueMatrix[i-1][j-1] = discount*(q*valueMatrix[i-1][j] + (1-q)*valueMatrix[i][j])
                
            if optionType:#american option pricing
                continiousValue = discount*(q*valueMatrix[i-1][j] + (1-q)*valueMatrix[i][j])
                exerciseValue = priceMatrix[i-1][j-1] - strikePrice
                valueMatrix[i-1][j-1] = max(exerciseValue, continiousValue)
                
    return valueMatrix

In [52]:
def bapm(curPrice, strikePrice, volatility, monthTerm, riskFree, numPeriods):
    #print('0: Put\n1: Call')
    
    call = 1
    while call not in [0, 1]:
        call = int(input('Select a 0 or 1: '))
    
    #print('\n0: European\n1: American')
    optionType = 0
    while optionType not in [0, 1]:
        optionType = int(input('Select a 0 or 1 for option type: '))
        
    start = time.clock()
    #setting variables
    timeStep = monthTerm/numPeriods
    #volatility = np.sqrt((volatility**2)/(12/timeStep))#scaling the volatility based on timeStep and months term for the option
    #not sure which time frame for vol to use, 3 months or 6 months
    u = np.exp(volatility/(np.sqrt(6)))
    d = u**-1
    q = (np.exp(riskFree*timeStep) - d)/(u-d)
    discount = np.exp(-riskFree*timeStep)
    
    priceMatrix = np.zeros((numPeriods+1, numPeriods+1))
    statePrices = np.zeros((numPeriods+1, numPeriods+1))
    
    for i in range(0, numPeriods + 1):#row loop
        for j in range(0, numPeriods + 1):#col loop
            #I chose to loop through the cols first and then rows
            statePrices = generate_statePrices(q=q, row=i, col=j, matrix=statePrices)
            priceMatrix = generate_stockPrices(u=u, d=d, row=i, col=j, matrix=priceMatrix)
    
    priceMatrix = curPrice*priceMatrix
    
    valueMatrix = np.zeros((numPeriods+1, numPeriods+1))
    valueMatrix[:, -1] = generate_finalValue(call=call, strikePrice=strikePrice, value=priceMatrix[:, -1])
    
    optionPrices = generate_optionPrices(optionType=optionType, strikePrice=strikePrice, valueMatrix=valueMatrix, q=q, discount=discount)
    
    #############################################################################################################
    #TESTING FUNCTIONS
    #print('Price Matrix:\n', priceMatrix, '\n')
    #print('Value Matrix:\n', valueMatrix)
    #print('State Prices Matrix:\n', statePrices, '\n')
    #print('Option Value --  State Prices Method --:\n', np.dot((discount*statePrices)**numPeriods, valueMatrix[:, -1]), '\n')
    print('Option Values -- backwards loop --:\n', optionPrices)
    #########################################################################################################
    end = time.clock()
    #print("Time Elapsed: ", end-start)
    
    return optionPrices[0][0]

In [19]:
bapm(strikePrice=40, curPrice=40, volatility=.3, monthTerm=6, riskFree=.04, numPeriods=5)

0: Put
1: Call
Select a 0 or 1: 1

0: European
1: American
Select a 0 or 1 for option type: 0
Price Matrix:
 [[40.         43.98056289 48.35724781 53.16947447 58.4605854  64.27823633]
 [ 0.         36.37970719 40.         43.98056289 48.35724781 53.16947447]
 [ 0.          0.         33.08707737 36.37970719 40.         43.98056289]
 [ 0.          0.          0.         30.09245466 33.08707737 36.37970719]
 [ 0.          0.          0.          0.         27.36886723 30.09245466]
 [ 0.          0.          0.          0.          0.         24.8917844 ]] 

State Prices Matrix:
 [[0.73506448 0.26493552 0.         0.         0.         0.        ]
 [0.         0.73506448 0.26493552 0.         0.         0.        ]
 [0.         0.         0.73506448 0.26493552 0.         0.        ]
 [0.         0.         0.         0.73506448 0.26493552 0.        ]
 [0.         0.         0.         0.         0.73506448 0.26493552]
 [0.         0.         0.         0.         0.         0.73506448]] 


8.977662990645424

In [55]:
def findVol(vol):
    calculatedPrice = bapm(strikePrice=40, curPrice=40, volatility=vol, monthTerm=6, riskFree=.04, numPeriods=2)
    realPrice = 4.03552
    
    return calculatedPrice - realPrice

In [56]:
parameters = [.3]
res = minimize(fun=findVol, x0=parameters)

Option Values -- backwards loop --:
 [[ 8.53577017  9.73479384 11.10224493]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Values -- backwards loop --:
 [[ 8.53577018  9.73479412 11.10224555]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Values -- backwards loop --:
 [[ 8.53577017  9.73479384 11.10224493]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Values -- backwards loop --:
 [[60.93593084  4.70436497  0.36318555]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Values -- backwards loop --:
 [[60.93593084  4.70436497  0.36318555]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Values -- backwards loop --:
 [[60.93585478  4.70436521  0.36318604]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Values -- backwards loop --:
 [[8.79711032 8.05504236 7.3755705

In [57]:
print('Parameters: ', res.x)

Parameters:  [0.29393878]
